In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

from langchain_groq import ChatGroq

os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")

llm=ChatGroq(model="llama-3.1-8b-instant")

result=llm.invoke("HEllo!")
result



AIMessage(content="Hello! It's nice to meet you. Is there something I can help you with or would you like to chat?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 39, 'total_tokens': 64, 'completion_time': 0.023442807, 'prompt_time': 0.002183347, 'queue_time': 0.052747675, 'total_time': 0.025626154}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_90c2e79dab', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--ba7f1d91-583d-4994-a902-a87f6ff15d96-0', usage_metadata={'input_tokens': 39, 'output_tokens': 25, 'total_tokens': 64})

In [2]:
from typing_extensions import TypedDict
from langgraph.graph import StateGraph,START,END
class State(TypedDict):
    topic:str
    character:str
    settings:str
    premises:str
    story_intro:str

In [3]:
def generate_characters(state:State):
    """Generate character description"""
    msg=llm.invoke(f"Create two character names and breif traits for a story about {state['topic']}")
    return {"character":msg.content}

def generate_setting(state:State):
    """Generate a story setting"""
    msg=llm.invoke(f"Describe a vivid settings for a story about {state['topic']}")
    return {"settings":msg.content}

def generate_premise(state:State):
    """Generate a story premise"""
    msg=llm.invoke(f"Write a one-sentence plot premise for a story about {state['topic']}")
    return {"premises":msg.content}

def combine_elements(state:State):
    msg=llm.invoke(
        f"Write a short introduction using these elements :\n"
        f"Character:{state['character']}\n"
        f"Settings:{state['settings']}\n"
        f"Premise:{state['premises']}"
    )
    return {"story_intro":msg.content}

In [4]:
graph=StateGraph(State)
graph.add_node("character",generate_characters)
graph.add_node("setting",generate_setting)
graph.add_node("premise",generate_premise)
graph.add_node("combine",combine_elements) 
from IPython.display import Image,display


In [5]:

graph.add_edge(START,"character")
graph.add_edge(START,"setting")
graph.add_edge(START,"premise")
graph.add_edge("character","combine")
graph.add_edge("setting","combine")
graph.add_edge("premise","combine")
graph.add_edge("combine",END)

compiled_graph= graph.compile()
graph_image=compiled_graph.get_graph().draw_mermaid_png()
display(Image(graph_image))

ValueError: Failed to reach https://mermaid.ink/ API while trying to render your graph. Status code: 502.

To resolve this issue:
1. Check your internet connection and try again
2. Try with higher retry settings: `draw_mermaid_png(..., max_retries=5, retry_delay=2.0)`
3. Use the Pyppeteer rendering method which will render your graph locally in a browser: `draw_mermaid_png(..., draw_method=MermaidDrawMethod.PYPPETEER)`

In [ ]:
state={"topic":"Space Adventure"}
res=compiled_graph.invoke(state)
res

NameError: name 'compiled_graph' is not defined

In [ ]:
print(res["character"])

In [ ]:
print(res["settings"])

In [ ]:
print(res['story_intro'])